In [1]:
# Use Pre-trained ResNet-18
# Image Classifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

# Data Transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

In [5]:
# Import data
data_dir = 'data/hymenoptera_data'
sets = ['train', 'val']
# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
#                   for x in ['train', 'val']}
image_datasets = {x: datasets.ImageFolder(data_dir+'/'+x, data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)

['ants', 'bees']


In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-'*10)

        # Train & Validation phase in each epoch
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                    
                    # Statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.4f}')
    print(f'Best Val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
# Fine tuning
# Load pre-trained model and reset final FC layer
# Pre-trained model을 불러와서 마지막 FC layer를 초기화한 후 전체 모델을 다시 학습하는 방식
model = models.resnet18(pretrained=True)
# To exchange last Fully Connected layer, get number of input features from the last layer 
num_ftrs = model.fc.in_features
# Exchange last Fully Connected layer output 10 -> 2
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion  = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Define scheduler to update learning rate
# step_size, gamma: Every {step_size} epoch, learning rate is multiply by {gamma}
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=25)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\moonc.DESKTOP-1BCQBNG/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


Epoch 0/24
----------
train Loss: 0.6917 Acc: 0.5943
val Loss: 0.4824 Acc: 0.8170
Epoch 1/24
----------
train Loss: 0.5322 Acc: 0.7582
val Loss: 0.3568 Acc: 0.9150
Epoch 2/24
----------
train Loss: 0.4877 Acc: 0.8074
val Loss: 0.2960 Acc: 0.9346
Epoch 3/24
----------
train Loss: 0.4396 Acc: 0.7951
val Loss: 0.2464 Acc: 0.9542
Epoch 4/24
----------
train Loss: 0.3971 Acc: 0.8361
val Loss: 0.2230 Acc: 0.9608
Epoch 5/24
----------
train Loss: 0.3686 Acc: 0.8443
val Loss: 0.2092 Acc: 0.9542
Epoch 6/24
----------
train Loss: 0.4012 Acc: 0.7910
val Loss: 0.1983 Acc: 0.9542
Epoch 7/24
----------
train Loss: 0.3390 Acc: 0.8607
val Loss: 0.1963 Acc: 0.9608
Epoch 8/24
----------
train Loss: 0.3229 Acc: 0.8730
val Loss: 0.2021 Acc: 0.9542
Epoch 9/24
----------
train Loss: 0.3572 Acc: 0.8648
val Loss: 0.2016 Acc: 0.9412
Epoch 10/24
----------
train Loss: 0.3385 Acc: 0.8197
val Loss: 0.1975 Acc: 0.9477
Epoch 11/24
----------
train Loss: 0.3474 Acc: 0.8525
val Loss: 0.1955 Acc: 0.9608
Epoch 12/24
--

In [9]:
# ConvNet as fixed feature extractor
# Freeze all network except final layer
# To freeze layers, set requires_grad=False
# Pre-trained model을 불러와 전체 모델을 freeze해서 모델 전체를 학습하지 않고, 마지막 FC layer만 학습하는 방식
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6909 Acc: 0.6516
val Loss: 0.3514 Acc: 0.8497
Epoch 1/24
----------
train Loss: 0.4670 Acc: 0.7828
val Loss: 0.2718 Acc: 0.8824
Epoch 2/24
----------
train Loss: 0.7220 Acc: 0.6844
val Loss: 0.2069 Acc: 0.9216
Epoch 3/24
----------
train Loss: 0.7123 Acc: 0.7459
val Loss: 0.5530 Acc: 0.7778
Epoch 4/24
----------
train Loss: 0.5621 Acc: 0.7582
val Loss: 0.1852 Acc: 0.9346
Epoch 5/24
----------
train Loss: 0.4340 Acc: 0.8238
val Loss: 0.3472 Acc: 0.8954
Epoch 6/24
----------
train Loss: 0.3687 Acc: 0.8402
val Loss: 0.1704 Acc: 0.9412
Epoch 7/24
----------
train Loss: 0.3548 Acc: 0.8607
val Loss: 0.1873 Acc: 0.9477
Epoch 8/24
----------
train Loss: 0.3117 Acc: 0.8525
val Loss: 0.1914 Acc: 0.9216
Epoch 9/24
----------
train Loss: 0.4110 Acc: 0.8402
val Loss: 0.2181 Acc: 0.9150
Epoch 10/24
----------
train Loss: 0.3714 Acc: 0.8402
val Loss: 0.1788 Acc: 0.9477
Epoch 11/24
----------
train Loss: 0.3547 Acc: 0.8361
val Loss: 0.2103 Acc: 0.9150
Epoch 12/24
--